In [ ]:
import time
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
from imagegenerator import HDR2SDRImageGenerator
from lutmaker import predict, write_lut_fast

In [ ]:
tf.config.list_physical_devices('GPU')

## Create the data generator

In [ ]:
image_map = []
imapf = open("../pyfilemap.txt", "r")

hdr = None
for line in imapf:
    if hdr is None:
        hdr = line.strip()
    else:
        image_map.append((hdr, line.strip()))
        hdr = None

imapf.close()

random.shuffle(image_map)

In [ ]:
image_gen = HDR2SDRImageGenerator(image_map, image_size=(1920, 1080), batch_size=2048, crop=(0, 0, 132, 132))

## Train the model

In [ ]:
def timestamp():
    return str(int(time.time()))


def build_model():
    model = keras.Sequential([
        layers.Dense(32, activation=tf.nn.relu, input_shape=[3]),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(3)
    ])
    
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
    
    # loss: mean_squared_error or mean_absolute_error
    model.compile(loss='mean_absolute_error',
                  optimizer=optimizer,
                  metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [ ]:
model = build_model()
display(model.summary())

In [ ]:
cp_filepath = "../checkpoints/weights-" + timestamp() + "-{loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(cp_filepath, monitor='loss')

model.fit(image_gen, epochs=50, callbacks=[checkpoint])

## Test and output LUT

In [ ]:
predict(model, 65535, 61937, 771) # The yellow of the CW logo - should be (133, 132, 81)